In [ ]:
from census import Census
import os 

# Get Api key from user env
c_apikey = os.environ.get("CENSUS_APIKEY")

if c_apikey:
    c = Census(c_apikey)
else:
    print("User did not set CENSUS_APIKEY in their environment")

: 

In [ ]:
#From each .csv file, only use the counties of interest

#Counties to include: 
#Loudoun County, Virginia
#Falls Church City, Virginia
#Alexandria City, Virginia
#Fairfax County, Virginia
#District of Columbia
#Fairfax City, Virginia
#Montgomery County, Maryland
#Prince William County, Virginia
#Prince Georges County, Maryland
#Arlington County, Virginia

import pandas as pd

# Define the list of DMV counties and cities
dmv_areas = [
    "Loudoun County, Virginia",
    "Falls Church City, Virginia",
    "Alexandria City, Virginia",
    "Fairfax County, Virginia",
    "District of Columbia",
    "Fairfax City, Virginia",
    "Montgomery County, Maryland",
    "Prince William County, Virginia",
    "Prince Georges County, Maryland",
    "Arlington County, Virginia"
]

# Read the four input files
file1 = "colonoscopy.csv" 
file2 = "household_income.csv"  
file3 = "SVI_index.csv"  
file4 = "college_educated.csv" 
file5 = "pap_smear.csv" 

# Load the datasets
data1 = pd.read_csv(file1, header=None)
data1.columns = ["County", "FIPS", "Model-Based Percent(3)", "Lower 95% Confidence Interval", "Upper 95% Confidence Interval"]

data2 = pd.read_csv(file2)
data2.columns = ["County", "FIPS", "Rural-Urban Continuum", "Value (Dollars)", "Rank within US"]

data3 = pd.read_csv(file3)
data3.columns = ["County", "FIPS", "Rural-Urban Continuum", "Value (Index)"]

data4 = pd.read_csv(file4)
data4.columns = ["County", "FIPS", "Rural-Urban Continuum", "Value (Percent)", "People (Education: At Least Bachelor's Degree)", "Rank within US"]

data5 = pd.read_csv(file5, header=None)
data1.columns = ["County", "FIPS", "Model-Based Percent(3)", "Lower 95% Confidence Interval", "Upper 95% Confidence Interval"]

# Filter the data for DMV counties and cities
filtered_data1 = data1[data1["County"].isin(dmv_areas)]
filtered_data2 = data2[data2["County"].isin(dmv_areas)]
filtered_data3 = data3[data3["County"].isin(dmv_areas)]
filtered_data4 = data4[data4["County"].isin(dmv_areas)]
filtered_data5 = data5[data5["County"].isin(dmv_areas)]

# Merge all four datasets on the 'County' column
merged_data = pd.merge(filtered_data1, filtered_data2, on="County", how="inner")
merged_data = pd.merge(merged_data, filtered_data3, on="County", how="inner")
merged_data = pd.merge(merged_data, filtered_data4, on="County", how="inner")
merged_data = pd.merge(merged_data, filtered_data5, on='County', how="inner")

# Save the merged data to a new CSV file
output_file = "merged_dmv_data.csv"
merged_data.to_csv(output_file, index=False)

print(f"Merged data saved to {output_file}")

